<a href="https://colab.research.google.com/github/NadirYasar/Customer_Churn_Odev2/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('CustomerChurn').getOrCreate()

In [5]:
data = spark.read.csv('/content/customer_churn.csv',inferSchema=True,header=True)
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
data.dtypes

[('Names', 'string'),
 ('Age', 'double'),
 ('Total_Purchase', 'double'),
 ('Account_Manager', 'int'),
 ('Years', 'double'),
 ('Num_Sites', 'double'),
 ('Onboard_date', 'timestamp'),
 ('Location', 'string'),
 ('Company', 'string'),
 ('Churn', 'int')]

In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [33]:
from pyspark.sql.functions import unix_timestamp, current_date, to_date

data = data.withColumn("Onboard_date", to_date("Onboard_date", "yyyy-MM-dd HH:mm:ss"))

data = data.withColumn(
    "Days_onboard",
    (unix_timestamp(current_date()) - unix_timestamp("Onboard_date")) / (60 * 60 * 24)
)
data.drop("Names","Location","Onboard_date","Company")
data.show()

+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+------------+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|            Location|             Company|Churn|Days_onboard|            features|
+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+------------+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|  2013-08-30|10265 Elizabeth M...|          Harvey LLC|    1|      4155.0|[42.0,11066.8,7.2...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|  2013-08-13|6157 Frank Garden...|          Wilson PLC|    1|      4172.0|[41.0,11916.22,6....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|  2016-06-29|1331 Keith Court ...|Miller, Johnson a...|    1|      3121.0|[38.0,12

In [10]:
data.select('Onboard_date').show()

+------------+
|Onboard_date|
+------------+
|  2013-08-30|
|  2013-08-13|
|  2016-06-29|
|  2014-04-22|
|  2016-01-19|
|  2009-03-03|
|  2016-12-05|
|  2006-03-09|
|  2011-09-29|
|  2006-03-28|
|  2016-11-13|
|  2015-05-28|
|  2011-02-16|
|  2012-11-22|
|  2015-03-28|
|  2015-07-22|
|  2006-09-03|
|  2006-10-22|
|  2015-10-07|
|  2014-11-06|
+------------+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import VectorAssembler
feature_cols= ["Age", "Total_Purchase", "Years", "Num_Sites", "Days_onboard","Account_Manager"]
assembler = VectorAssembler(inputCols = feature_cols, outputCol= 'features1')
output = assembler.transform(data)
data = data.withColumnRenamed("features", "features1")
final_data = output.select("features1","Churn")
final_data.show()

+--------------------+-----+
|           features1|Churn|
+--------------------+-----+
|[42.0,11066.8,7.2...|    1|
|[41.0,11916.22,6....|    1|
|[38.0,12884.75,6....|    1|
|[42.0,8010.76,6.7...|    1|
|[37.0,9191.58,5.5...|    1|
|[48.0,10356.02,5....|    1|
|[44.0,11331.58,5....|    1|
|[32.0,9885.12,6.9...|    1|
|[43.0,14062.6,5.4...|    1|
|[40.0,8066.94,7.1...|    1|
|[30.0,11575.37,5....|    1|
|[45.0,8771.02,6.6...|    1|
|[45.0,8988.67,4.8...|    1|
|[40.0,8283.32,5.1...|    1|
|[41.0,6569.87,4.3...|    1|
|[38.0,10494.82,6....|    1|
|[45.0,8213.41,7.3...|    1|
|[43.0,11226.88,8....|    1|
|[53.0,5515.09,6.8...|    1|
|[46.0,8046.4,5.69...|    1|
+--------------------+-----+
only showing top 20 rows



In [12]:
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [13]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features1', labelCol='Churn')
lrm= lr.fit(train_data)
lrm_summary = lrm.summary
lrm.summary.accuracy

0.9023383768913342

In [14]:
test_data.show()

+--------------------+-----+
|           features1|Churn|
+--------------------+-----+
|[27.0,8628.8,5.3,...|    0|
|[28.0,11128.95,5....|    0|
|[28.0,11245.38,6....|    0|
|[29.0,5900.78,5.5...|    0|
|[29.0,8688.17,5.7...|    1|
|[29.0,10203.18,5....|    0|
|[31.0,10058.87,6....|    0|
|[31.0,11297.57,6....|    1|
|[31.0,12264.68,5....|    0|
|[32.0,12254.75,5....|    0|
|[33.0,4711.89,4.9...|    0|
|[33.0,10309.71,2....|    0|
|[33.0,12115.91,5....|    0|
|[33.0,12638.51,5....|    0|
|[33.0,14160.05,4....|    0|
|[34.0,6461.86,4.6...|    0|
|[34.0,8772.26,5.7...|    0|
|[34.0,9270.9,6.19...|    0|
|[34.0,9779.12,5.8...|    0|
|[34.0,10674.92,3....|    0|
+--------------------+-----+
only showing top 20 rows



In [15]:
lrm.summary.areaUnderROC

0.9138399827706906

In [16]:
lrm.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|           features1|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|  0.0|[5.23831355182825...|[0.99471883578699...|       0.0|
|[25.0,9672.03,5.4...|  0.0|[5.02849659941029...|[0.99349395737028...|       0.0|
|[26.0,8787.39,5.4...|  1.0|[0.93449961683774...|[0.71798726787696...|       0.0|
|[26.0,8939.61,4.5...|  0.0|[6.84769282135851...|[0.99893922357312...|       0.0|
|[28.0,8670.98,3.9...|  0.0|[8.27773081249401...|[0.99974595138036...|       0.0|
|[28.0,9090.43,5.7...|  0.0|[1.87910749327782...|[0.86750857765854...|       0.0|
|[28.0,11204.23,3....|  0.0|[1.97116107709670...|[0.87773576962267...|       0.0|
|[29.0,9378.24,4.9...|  0.0|[5.28807721814171...|[0.99497392524389...|       0.0|
|[29.0,9617.59,5.4...|  0.0|[4.90695302641085...|[0.99265929802437...|       0.0|
|[29.0,11274.46,

In [17]:
lrm.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                727|                727|
|   mean|0.16918844566712518|0.12379642365887207|
| stddev|0.37517639778255024| 0.3295759063688022|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

prediction_test = lrm.evaluate(test_data)
prediction_test.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|           features1|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,5.3,...|    0|[6.18475739414894...|[0.99794363594071...|       0.0|
|[28.0,11128.95,5....|    0|[4.58816959538044...|[0.98993095762343...|       0.0|
|[28.0,11245.38,6....|    0|[4.35343960189524...|[0.98730084822507...|       0.0|
|[29.0,5900.78,5.5...|    0|[4.46749434653778...|[0.98865417031458...|       0.0|
|[29.0,8688.17,5.7...|    1|[3.16202807888692...|[0.95938005426769...|       0.0|
|[29.0,10203.18,5....|    0|[4.30309383081932...|[0.98665388312887...|       0.0|
|[31.0,10058.87,6....|    0|[4.98600811238862...|[0.99321348524632...|       0.0|
|[31.0,11297.57,6....|    1|[1.29307115298792...|[0.78466656004140...|       0.0|
|[31.0,12264.68,5....|    0|[4.01634771214889...|[0.98230027107498...|       0.0|
|[32.0,12254.75,

In [19]:
evaluator =BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Churn")
auc = evaluator.evaluate(prediction_test.predictions)
auc

0.7874175545408423

In [20]:
from pyspark.ml.classification import RandomForestClassifier


rf = RandomForestClassifier(labelCol="Churn", featuresCol="features1", numTrees=100, maxDepth=10)


rf_model = rf.fit(train_data)

rf_predictions = rf_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="Churn", rawPredictionCol="prediction", metricName="areaUnderROC")
auc_rf = evaluator.evaluate(rf_predictions)
print(f"AUC (Random Forest): {auc_rf}")


AUC (Random Forest): 0.7030695078640283


In [21]:
from pyspark.ml.classification import GBTClassifier


gbt = GBTClassifier(labelCol="Churn", featuresCol="features1", maxIter=100)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)
auc_gbt = evaluator.evaluate(gbt_predictions)
print(f"AUC (Gradient Boosting): {auc_gbt}")

AUC (Gradient Boosting): 0.6763064434297311


In [22]:
from pyspark.ml.classification import LogisticRegression

# Logistic Regression modeli
lr = LogisticRegression(labelCol="Churn", featuresCol="features1", regParam=0.01, elasticNetParam=0.8)

# Modeli eğitme
lr_model = lr.fit(train_data)

# Modeli test verisiyle değerlendirme
lr_predictions = lr_model.transform(test_data)

# AUC değerlendirmesi
auc_lr = evaluator.evaluate(lr_predictions)
print(f"AUC (Logistic Regression with Regularization): {auc_lr}")


AUC (Logistic Regression with Regularization): 0.7874175545408423


In [23]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Days_onboard: double (nullable = true)



In [26]:
# XGBoost modelinin tahminlerini yapma
predictions = rf_model.transform(test_data)

# Tahminler ve gerçek değerleri gösterme
predictions.select("prediction", "Churn", "features1").show()

# İleri düzey sonuçlar (probability vs rawPrediction)
predictions.select("prediction", "rawPrediction", "probability").show()


+----------+-----+--------------------+
|prediction|Churn|           features1|
+----------+-----+--------------------+
|       0.0|    0|[27.0,8628.8,5.3,...|
|       0.0|    0|[28.0,11128.95,5....|
|       0.0|    0|[28.0,11245.38,6....|
|       0.0|    0|[29.0,5900.78,5.5...|
|       0.0|    1|[29.0,8688.17,5.7...|
|       0.0|    0|[29.0,10203.18,5....|
|       0.0|    0|[31.0,10058.87,6....|
|       0.0|    1|[31.0,11297.57,6....|
|       0.0|    0|[31.0,12264.68,5....|
|       0.0|    0|[32.0,12254.75,5....|
|       0.0|    0|[33.0,4711.89,4.9...|
|       0.0|    0|[33.0,10309.71,2....|
|       0.0|    0|[33.0,12115.91,5....|
|       0.0|    0|[33.0,12638.51,5....|
|       0.0|    0|[33.0,14160.05,4....|
|       0.0|    0|[34.0,6461.86,4.6...|
|       0.0|    0|[34.0,8772.26,5.7...|
|       0.0|    0|[34.0,9270.9,6.19...|
|       0.0|    0|[34.0,9779.12,5.8...|
|       0.0|    0|[34.0,10674.92,3....|
+----------+-----+--------------------+
only showing top 20 rows

+----------+--

In [28]:
new_customers_data = spark.read.csv('/content/new_customers.csv',inferSchema=True,header=True)
new_customers_data.show()

+-----------------+---------------+----------------+---------------+-----+---------+-------------------+--------------------+-------+
|            Names|            Age|  Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|Company|
+-----------------+---------------+----------------+---------------+-----+---------+-------------------+--------------------+-------+
|    Andrew Mccall|           37.0|         9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|   NULL|
|      Nataliebury| WI 15717-8316"|        King Ltd|           NULL| NULL|     NULL|               NULL|                NULL|   NULL|
|   Michele Wright|           23.0|         7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   NULL|
|        Youngport| ME 23686-4381"|   Cannon-Benson|           NULL| NULL|     NULL|               NULL|                NULL|   NULL|
|     Jeremy Chang|           65.0|           100.0|          

In [29]:
new_customers_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company']

In [30]:
new_customers_data.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|          Location|Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-------+
|  count|           12|                12|               11|                 6|                6|                 6|                 6|      0|
|   mean|         NULL|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|              NULL|   NULL|
| stddev|         NULL| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|              NULL|   NULL|
|    min|Andrew Mccall|    ME 23686-4381"|            100.0|                 0|              1.0|               8.0|  085 Austin Views| 

In [31]:
new_customers_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Total_Purchase: string (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [35]:
from pyspark.sql.functions import unix_timestamp, current_date, to_date

new_customers_data = data.withColumn("Onboard_date", to_date("Onboard_date", "yyyy-MM-dd HH:mm:ss"))

new_customers_data = data.withColumn(
    "Days_onboard",
    (unix_timestamp(current_date()) - unix_timestamp("Onboard_date")) / (60 * 60 * 24)
)
new_customers_data.drop("Names","Location","Onboard_date","Company")
new_customers_data.show()

+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+------------+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|            Location|             Company|Churn|Days_onboard|            features|
+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+------------+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|  2013-08-30|10265 Elizabeth M...|          Harvey LLC|    1|      4155.0|[42.0,11066.8,7.2...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|  2013-08-13|6157 Frank Garden...|          Wilson PLC|    1|      4172.0|[41.0,11916.22,6....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|  2016-06-29|1331 Keith Court ...|Miller, Johnson a...|    1|      3121.0|[38.0,12

In [40]:
from pyspark.ml.feature import VectorAssembler
feature_cols= ["Age", "Total_Purchase", "Years", "Num_Sites", "Days_onboard","Account_Manager"]
assembler = VectorAssembler(inputCols = feature_cols, outputCol= 'features2')
output = assembler.transform(new_customers_data)
data = data.withColumnRenamed("features", "features2")
final_data = output.select("features2","Churn")
final_data.show()

+--------------------+-----+
|           features2|Churn|
+--------------------+-----+
|[42.0,11066.8,7.2...|    1|
|[41.0,11916.22,6....|    1|
|[38.0,12884.75,6....|    1|
|[42.0,8010.76,6.7...|    1|
|[37.0,9191.58,5.5...|    1|
|[48.0,10356.02,5....|    1|
|[44.0,11331.58,5....|    1|
|[32.0,9885.12,6.9...|    1|
|[43.0,14062.6,5.4...|    1|
|[40.0,8066.94,7.1...|    1|
|[30.0,11575.37,5....|    1|
|[45.0,8771.02,6.6...|    1|
|[45.0,8988.67,4.8...|    1|
|[40.0,8283.32,5.1...|    1|
|[41.0,6569.87,4.3...|    1|
|[38.0,10494.82,6....|    1|
|[45.0,8213.41,7.3...|    1|
|[43.0,11226.88,8....|    1|
|[53.0,5515.09,6.8...|    1|
|[46.0,8046.4,5.69...|    1|
+--------------------+-----+
only showing top 20 rows



In [41]:
new_customers = assembler.transform(new_customers_data)
new_customers.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Days_onboard: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- features2: vector (nullable = true)



In [42]:
train1_data, test1_data = final_data.randomSplit([0.8,0.2])

In [44]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features2', labelCol='Churn')
lrm= lr.fit(train1_data)
lrm_summary = lrm.summary
lrm.summary.accuracy

0.8871428571428571

In [45]:
lrm.summary.areaUnderROC

0.9049899193548473

In [47]:
lrm.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|           features2|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|  0.0|[4.03509698508933...|[0.98262332358752...|       0.0|
|[25.0,9672.03,5.4...|  0.0|[4.12159417418358...|[0.98404020761574...|       0.0|
|[26.0,8787.39,5.4...|  1.0|[0.16240421981829...|[0.54051205151763...|       0.0|
|[26.0,8939.61,4.5...|  0.0|[5.81893471512175...|[0.99703803114866...|       0.0|
|[27.0,8628.8,5.3,...|  0.0|[4.89642907369241...|[0.99258221296333...|       0.0|
|[28.0,8670.98,3.9...|  0.0|[7.22543057123936...|[0.99927269027940...|       0.0|
|[28.0,9090.43,5.7...|  0.0|[1.07107483521335...|[0.74480126602091...|       0.0|
|[28.0,11128.95,5....|  0.0|[3.69753687901405...|[0.97581491659769...|       0.0|
|[28.0,11204.23,3....|  0.0|[1.55389128819874...|[0.82547504434777...|       0.0|
|[28.0,11245.38,